In [89]:
#!/usr/bin/env python3
"""
3D Milling Environment with Random Moves + Path Trail
-----------------------------------------------------
We have a half-sphere shape anchored on z=0 (no overhang),
a block of stock, and a router that moves in 3D.

We do a random series of moves: for each move, we pick a
random coordinate in-bounds. We attempt to remove stock
along the path from current router pos -> new coordinate.
If that path intersects the shape or out-of-bounds, the run
ends. Otherwise, we remove outside stock.

At the end, we visualize the final environment with a scatter
plot. We also draw a black line that traces the router's movement
trail throughout the random moves.

This is purely a demonstration: no RL yet, just random "game movement."
"""
import numpy as np
import matplotlib.pyplot as plt

class Milling3DEnvNoOverhang:
    def __init__(
        self,
        grid_size=20,
        min_radius=3,
        max_radius=6,
        max_steps=30
    ):
        """
        :param grid_size: NxNxN dimension
        :param min_radius,max_radius: radius range for the half-sphere
        :param max_steps: maximum random moves before we end
        """
        self.grid_size = grid_size
        self.min_radius = min_radius
        self.max_radius = max_radius
        self.max_steps = max_steps

        self.reset()

    def reset(self):
        """
        1) stock: NxNxN => True means voxel present
        2) shape: NxNxN => half-sphere anchored at z=0
        3) router_pos => (0,0,0)
        4) router_path => list of positions visited
        5) moves_count => how many successful moves before done
        6) total_removed => how many stock voxels were removed in total
        """
        n = self.grid_size
        self.stock = np.ones((n,n,n), dtype=bool)
        self.shape = np.zeros((n,n,n), dtype=bool)

        # random radius
        r = np.random.randint(self.min_radius, self.max_radius+1)
        # pick random cx, cy but anchor z=0
        cx = np.random.randint(r, n - r)
        cy = np.random.randint(r, n - r)
        cz = 0

        # fill shape
        for x in range(n):
            for y in range(n):
                for z in range(n):
                    dx = x - cx
                    dy = y - cy
                    dz = z - cz
                    # half-sphere for z>=0
                    if z >= 0:
                        dist_sq = dx*dx + dy*dy + dz*dz
                        if dist_sq <= r*r:
                            self.shape[x,y,z] = True

        self.router_pos = np.array([0,0,0], dtype=int)
        self.steps_taken = 0
        self.done = False

        # Path of router
        self.router_path = [tuple(self.router_pos)]
        
        # Stats
        self.moves_count = 0
        self.total_removed = 0

    def line_voxels(self, start, end):
        """
        Return the integer voxel coords along line from start->end.
        A naive float-based approach for demonstration.
        """
        s = start.astype(float)
        e = end.astype(float)
        diff = e - s
        length = int(np.linalg.norm(diff))
        if length == 0:
            return [tuple(start)]
        steps = max(1, length*2)
        out = []
        visited = set()
        for i in range(steps+1):
            t = i/steps
            coords = np.round(s + diff*t).astype(int)
            c_tuple = tuple(coords)
            if c_tuple not in visited:
                visited.add(c_tuple)
                out.append(c_tuple)
        return out

    def do_move(self, target):
        """
        Attempt to move from self.router_pos -> target.
        If path intersects shape or out-of-bounds, we end (fail).
        Otherwise, remove outside stock along path.
        We also update self.moves_count and self.total_removed.
        """
        if self.done:
            return
        
        path_voxels = self.line_voxels(self.router_pos, target)
        failed = False
        removed_this_move = 0

        for vx, vy, vz in path_voxels:
            if not (0 <= vx < self.grid_size and 0 <= vy < self.grid_size and 0 <= vz < self.grid_size):
                # out-of-bounds
                failed = True
                break
            if self.shape[vx,vy,vz]:
                # hitting shape
                failed = True
                break
            # remove stock if present
            if self.stock[vx,vy,vz]:
                self.stock[vx,vy,vz] = False
                removed_this_move += 1

        if not failed:
            self.router_pos = target.copy()
            self.router_path.append(tuple(self.router_pos))
            self.moves_count += 1
            self.total_removed += removed_this_move
        else:
            self.done = True

        self.steps_taken += 1
        if self.steps_taken >= self.max_steps:
            self.done = True

    def random_playthrough(self):
        """
        Keep picking random target coords until we fail or max_steps.
        """
        while not self.done:
            tx = np.random.randint(0, self.grid_size)
            ty = np.random.randint(0, self.grid_size)
            tz = np.random.randint(0, self.grid_size)
            self.do_move(np.array([tx,ty,tz], dtype=int))

    def visualize(self, elev=30, azim=45, title="Final State"):
        """
        3D scatter plot:
         - shape & stock => bright blue
         - outside & stock => gray
         - partial alpha
         - black line for path
         - big red dot for final router pos
        Also print out # of moves & total_removed in the console.
        """
        n = self.grid_size
        stock, shape = self.stock, self.shape

        shape_x, shape_y, shape_z = [], [], []
        outside_x, outside_y, outside_z = [], [], []

        for x in range(n):
            for y in range(n):
                for z in range(n):
                    if stock[x,y,z]:
                        if shape[x,y,z]:
                            shape_x.append(x)
                            shape_y.append(y)
                            shape_z.append(z)
                        else:
                            outside_x.append(x)
                            outside_y.append(y)
                            outside_z.append(z)

        fig = plt.figure(figsize=(10,8))
        ax = fig.add_subplot(111, projection='3d')

        # shape
        ax.scatter(shape_x, shape_y, shape_z, c='blue', alpha=0.4, s=20, marker='o', label='Shape')
        # outside
        ax.scatter(outside_x, outside_y, outside_z, c='gray', alpha=0.2, s=20, marker='o', label='Outside')

        # path => black line
        if len(self.router_path) > 1:
            px = [p[0]+0.5 for p in self.router_path]
            py = [p[1]+0.5 for p in self.router_path]
            pz = [p[2]+0.5 for p in self.router_path]
            ax.plot(px, py, pz, c='black', linewidth=2, label='Path')

        # final router pos => big red dot
        rx, ry, rz = self.router_pos
        ax.scatter(rx+0.5, ry+0.5, rz+0.5, c='red', s=200, alpha=1.0, marker='o', label='Router')

        ax.set_xlim(0,n)
        ax.set_ylim(0,n)
        ax.set_zlim(0,n)
        ax.view_init(elev=elev, azim=azim)
        ax.set_title(title)
        ax.legend(loc='upper right')
        plt.tight_layout()
        plt.show()

        # print stats
        print(f"Number of moves made: {self.moves_count}")
        print(f"Total stock voxels removed: {self.total_removed}")

def random_play_through():
    env = Milling3DEnv(grid_size=20, min_radius=3, max_radius=6, max_steps=15)
    # random playthrough
    env.random_playthrough()
    # visualize final result
    env.visualize(elev=30, azim=45, title="Random Moves Final State")

In [104]:
"""
Minimal DQN for Milling Environment with Reward Normalization and TensorBoard Logging
---------------------------------------------------------------------------------------
This script implements a minimal DQN applied to our 3D milling environment (no-overhang half-sphere).
The environment simulates:
  - A voxel grid representing stock (True means present, False removed)
  - A protected target shape (a half-sphere anchored on z=0)
  - A router whose moves remove stock along a path; if the path touches the target or goes OOB, the episode ends.

The DQN is trained with a replay buffer, and we log multiple metrics (episode reward, steps, stock removed,
and the loss) to TensorBoard. Reward normalization is performed using an exponential moving average.
"""

import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
from torch.utils.tensorboard import SummaryWriter
import datetime
import os

# -------------------------------------------------------------------------
# 1) Environment: Milling3DEnvNoOverhang
# -------------------------------------------------------------------------

class Milling3DEnvNoOverhang:
    def __init__(self, grid_size=8, min_radius=2, max_radius=3, max_steps=50):
        self.grid_size = grid_size
        self.min_radius = min_radius
        self.max_radius = max_radius
        self.max_steps = max_steps
        self.reset()

    def reset(self):
        n = self.grid_size
        # Initialize stock: all voxels present
        self.stock = np.ones((n, n, n), dtype=bool)
        # Initialize shape: will be a half-sphere anchored at z=0
        self.shape = np.zeros((n, n, n), dtype=bool)

        # Random radius for half-sphere
        r = np.random.randint(self.min_radius, self.max_radius + 1)
        # Choose center (cx,cy) such that shape fits, and set cz=0 (bottom)
        cx = np.random.randint(r, n - r)
        cy = np.random.randint(r, n - r)
        cz = 0

        for x in range(n):
            for y in range(n):
                for z in range(n):
                    if z >= 0:
                        dx = x - cx
                        dy = y - cy
                        dz = z - cz
                        if dx*dx + dy*dy + dz*dz <= r*r:
                            self.shape[x, y, z] = True

        # Set initial router position
        self.router_pos = np.array([0, 0, 0], dtype=int)
        self.steps_taken = 0
        self.done = False
        self.total_removed = 0.0
        self.termination_reason = None

        return self._get_observation()

    def line_voxels(self, start, end):
        s = start.astype(float)
        e = end.astype(float)
        diff = e - s
        length = int(np.linalg.norm(diff))
        if length == 0:
            return [tuple(start)]
        steps = max(1, length * 2)
        visited = set()
        out = []
        for i in range(steps + 1):
            t = i / steps
            coords = np.round(s + diff * t).astype(int)
            c_tuple = tuple(coords)
            if c_tuple not in visited:
                visited.add(c_tuple)
                out.append(c_tuple)
        return out

    def do_move(self, target):
        if self.done:
            return self._get_observation(), 0.0, True

        path_vox = self.line_voxels(self.router_pos, target)
        failed = False
        reward = 0.0
        reason = None

        amount_removed = 0.0
        for vx, vy, vz in path_vox:
            if not (0 <= vx < self.grid_size and 0 <= vy < self.grid_size and 0 <= vz < self.grid_size):
                failed = True
                #reward -= 100.0
                reason = "out_of_bounds"
                break
            if self.shape[vx, vy, vz]:
                failed = True
                #reward -= 100.0
                reason = "shape"
                break
            if self.stock[vx, vy, vz]:
                self.stock[vx, vy, vz] = False
                amount_removed += 1.0
                self.total_removed += 1.0
            
        # update reward scaling based on amount removed
        if amount_removed > 0:
            percent_removed = amount_removed / np.sum(self.stock)
            reward += percent_removed * 100.0

        if not failed:
            #reward -= 2 # step cost
            self.router_pos = target.copy()

        self.steps_taken += 1
        if self.steps_taken >= self.max_steps and not failed:
            failed = True
            reason = "max_steps"

        # If all outside stock removed, success
        outside_mask = (self.shape == 0)
        if not failed and np.sum(self.stock[outside_mask]) == 0:
            reward += 200.0  # success bonus
            failed = True
            reason = "complete"

        if failed:
            self.done = True
            self.termination_reason = reason

        return self._get_observation(), reward, self.done

    def _get_observation(self):
        # Flatten stock (0/1), shape (0/1), and router coordinates (3 numbers)
        stock_f = self.stock.flatten().astype(np.float32)
        shape_f = self.shape.flatten().astype(np.float32)
        router_f = self.router_pos.astype(np.float32)
        return np.concatenate([stock_f, shape_f, router_f], axis=0)

    def step(self, action):
        # Decode action from [0, grid_size^3-1] to (x,y,z)
        n = self.grid_size
        z = action % n
        y = (action // n) % n
        x = (action // (n * n)) % n
        next_obs, r, done = self.do_move(np.array([x, y, z], dtype=int))
        return next_obs, r, done, {}

    def reset_gym(self):
        return self.reset()

    @property
    def n_actions(self):
        return self.grid_size**3

    def fraction_outside_removed(self):
        outside_mask = (self.shape == 0)
        current_outside = np.sum(self.stock[outside_mask])
        removed = self.total_removed  # since every removed voxel from outside increments total_removed
        total = np.sum(outside_mask)
        return removed / (total + 1e-8)

# -------------------------------------------------------------------------
# 2) Reward Normalizer
# -------------------------------------------------------------------------

class RewardNormalizer:
    def __init__(self, momentum=0.99, eps=1e-8):
        self.momentum = momentum
        self.eps = eps
        self.mean = 0.0
        self.var = 1.0
        self.count = 0

    def update(self, r):
        self.mean = self.momentum * self.mean + (1 - self.momentum) * r
        self.var = self.momentum * self.var + (1 - self.momentum) * ((r - self.mean) ** 2)
        self.count += 1

    def normalize(self, r):
        return (r - self.mean) / (np.sqrt(self.var) + self.eps)

# -------------------------------------------------------------------------
# 3) Simple DQN and Replay Buffer
# -------------------------------------------------------------------------

class SimpleDQN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=256):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
    def forward(self, x):
        return self.net(x)

class ReplayBuffer:
    def __init__(self, capacity=10000):
        self.buffer = deque(maxlen=capacity)
    def push(self, s, a, r, s_next, done):
        self.buffer.append((s, a, r, s_next, done))
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        s, a, r, s_next, done = zip(*batch)
        return np.array(s), np.array(a), np.array(r), np.array(s_next), np.array(done)
    def __len__(self):
        return len(self.buffer)

# -------------------------------------------------------------------------
# 4) DQN Training Loop with TensorBoard Logging and Reward Normalization
# -------------------------------------------------------------------------

def train_dqn(env, num_episodes=200, batch_size=32, gamma=0.99, lr=1e-3,
              epsilon_start=1.0, epsilon_end=0.1, epsilon_decay=0.995, replay_capacity=10000):
    device = "cpu"
    obs_dim = len(env.reset_gym())
    n_actions = env.n_actions

    dqn = SimpleDQN(obs_dim, n_actions).to(device)
    optimizer = optim.Adam(dqn.parameters(), lr=lr)
    replay = ReplayBuffer(capacity=replay_capacity)

    # Initialize reward normalizer
    reward_normalizer = RewardNormalizer(momentum=0.99)

    # TensorBoard setup
    log_dir = os.path.join("runs", f"dqn_milling_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}")
    writer = SummaryWriter(log_dir=log_dir)

    epsilon = epsilon_start
    global_step = 0
    update_count = 0

    # Counters for termination reasons
    term_counters = {"shape": 0, "out_of_bounds": 0, "max_steps": 0, "complete": 0}

    for ep in range(num_episodes):
        obs = env.reset_gym()
        obs_t = torch.FloatTensor(obs).to(device)
        done = False
        total_reward = 0.0
        steps_this_ep = 0

        while not done:
            global_step += 1
            # Epsilon-greedy selection
            if random.random() < epsilon:
                action = random.randint(0, n_actions - 1)
            else:
                with torch.no_grad():
                    q_vals = dqn(obs_t.unsqueeze(0))
                    action = q_vals.argmax(dim=1).item()

            next_obs, reward, done, _ = env.step(action)
            # Update the reward normalizer and normalize reward
            reward_normalizer.update(reward)
            norm_reward = reward_normalizer.normalize(reward)
            total_reward += reward

            replay.push(obs, action, norm_reward, next_obs, done)

            obs = next_obs
            obs_t = torch.FloatTensor(obs).to(device)
            steps_this_ep += 1

            if len(replay) >= batch_size:
                s_arr, a_arr, r_arr, s_next_arr, d_arr = replay.sample(batch_size)
                s_ten = torch.FloatTensor(s_arr).to(device)
                a_ten = torch.LongTensor(a_arr).to(device)
                r_ten = torch.FloatTensor(r_arr).to(device)
                s_next_ten = torch.FloatTensor(s_next_arr).to(device)
                d_ten = torch.BoolTensor(d_arr).to(device)

                q_vals = dqn(s_ten)
                q_s_a = q_vals.gather(1, a_ten.unsqueeze(1)).squeeze(1)

                with torch.no_grad():
                    q_next = dqn(s_next_ten)
                    max_q_next, _ = torch.max(q_next, dim=1)
                    max_q_next[d_ten] = 0.0
                target = r_ten + gamma * max_q_next

                loss = nn.MSELoss()(q_s_a, target)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                update_count += 1
                writer.add_scalar("Loss", loss.item(), update_count)

        # End of episode: epsilon decay
        epsilon = max(epsilon_end, epsilon * epsilon_decay)

        # Log additional metrics:
        writer.add_scalar("EpisodeReward", total_reward, ep)
        writer.add_scalar("StepsPerEpisode", steps_this_ep, ep)
        writer.add_scalar("StockRemoved", env.total_removed, ep)
        outside_mask = (env.shape == 0)
        outside_total = np.sum(outside_mask)
        frac_removed = env.total_removed / (outside_total + 1e-8)
        writer.add_scalar("FractionRemoved", frac_removed, ep)

        # Track termination reason
        term_reason = env.termination_reason if env.termination_reason is not None else "max_steps"
        if term_reason in term_counters:
            term_counters[term_reason] += 1
        writer.add_scalar(f"Terminations/{term_reason}", term_counters[term_reason], ep)

        print(f"Episode {ep+1}/{num_episodes}: Reward={total_reward:.2f}, Steps={steps_this_ep}, "
              f"Epsilon={epsilon:.2f}, StockRemoved={env.total_removed:.0f}, FractionRemoved={frac_removed:.3f}, "
              f"Termination={term_reason}")

    writer.close()
    print("Training complete.")
    return dqn

# -------------------------------------------------------------------------
# 5) Testing Routine
# -------------------------------------------------------------------------

def test_dqn(dqn, env, num_tests=100):
    """
    Runs the trained DQN for num_tests episodes using a greedy (epsilon=0) policy,
    and returns the average fraction of outside stock removed (as a percentage)
    and the average number of steps per episode.
    """
    device = "cpu"
    total_frac = 0.0
    total_steps = 0

    for test in range(num_tests):
        obs = env.reset_gym()
        done = False
        steps = 0

        while not done:
            obs_t = torch.FloatTensor(obs).to(device)
            with torch.no_grad():
                q_vals = dqn(obs_t.unsqueeze(0))
                action = q_vals.argmax(dim=1).item()
            obs, reward, done, _ = env.step(action)
            steps += 1

        frac_removed = env.fraction_outside_removed()  # returns fraction (0 to 1)
        total_frac += frac_removed
        total_steps += steps

    avg_frac = total_frac / num_tests
    avg_steps = total_steps / num_tests
    print(f"Average fraction of outside stock removed: {avg_frac * 100:.2f}%")
    print(f"Average number of steps per episode: {avg_steps:.2f}")
    return avg_frac, avg_steps

In [ ]:
env = Milling3DEnvNoOverhang(grid_size=8, min_radius=2, max_radius=3, max_steps=30)
trained_dqn = train_dqn(env, num_episodes=10000, batch_size=128, gamma=0.1,
            lr=1e-3, epsilon_start=1.0, epsilon_end=0.3, epsilon_decay=0.99995,
            replay_capacity=10000)
# Visualize the final environment
#env.visualize(elev=30, azim=45, title="Final State After Training")
#env.visualize(elev=30, azim=45, title="Router Path", show_path=True)

Episode 1/5000: Reward=3.60, Steps=3, Epsilon=1.00, StockRemoved=18, FractionRemoved=0.041, Termination=shape
Episode 2/5000: Reward=14.79, Steps=14, Epsilon=1.00, StockRemoved=70, FractionRemoved=0.143, Termination=shape
Episode 3/5000: Reward=6.70, Steps=6, Epsilon=1.00, StockRemoved=33, FractionRemoved=0.067, Termination=shape
Episode 4/5000: Reward=1.79, Steps=2, Epsilon=1.00, StockRemoved=9, FractionRemoved=0.021, Termination=shape
Episode 5/5000: Reward=1.19, Steps=1, Epsilon=1.00, StockRemoved=6, FractionRemoved=0.012, Termination=shape
Episode 6/5000: Reward=7.98, Steps=7, Epsilon=1.00, StockRemoved=39, FractionRemoved=0.080, Termination=shape
Episode 7/5000: Reward=0.39, Steps=1, Epsilon=1.00, StockRemoved=2, FractionRemoved=0.005, Termination=shape
Episode 8/5000: Reward=0.39, Steps=1, Epsilon=1.00, StockRemoved=2, FractionRemoved=0.005, Termination=shape
Episode 9/5000: Reward=0.59, Steps=1, Epsilon=1.00, StockRemoved=3, FractionRemoved=0.007, Termination=shape
Episode 10/50

In [110]:
test_dqn(trained_dqn, env, num_tests=100)

Average fraction of outside stock removed: 15.74%
Average number of steps per episode: 22.09


(0.1573785200993817, 22.09)

In [92]:
!tensorboard --logdir=runs

TensorFlow installation not found - running with reduced feature set.
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.19.0 at http://localhost:6006/ (Press CTRL+C to quit)


OSError: [Errno 5] Input/output error

In [93]:
random_play_through()

NameError: name 'Milling3DEnv' is not defined